<a href="https://colab.research.google.com/github/lanzizuan/TSP-Parallelization/blob/master/TSP-parallelization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#get distance file from github
!wget https://raw.githubusercontent.com/lanzizuan/Numbers/master/path.py

--2019-03-02 16:14:05--  https://raw.githubusercontent.com/lanzizuan/Numbers/master/path.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 79021 (77K) [text/plain]
Saving to: ‘path.py’

path.py             100%[===================>]  77.17K  --.-KB/s    in 0.02s   

2019-03-02 16:14:05 (3.80 MB/s) - ‘path.py’ saved [79021/79021]



In [0]:
#!/usr/bin/env python
"""
Traveling salesman solution with random start and greedy path selection and parallelization

"""
#import distance between 50 capitals of US states
from path import values

#import packages
import sys
from random import choice
import numpy as np
import pandas as pd
from multiprocessing import Pool
import warnings
warnings.filterwarnings('ignore')


In [0]:
dt = np.dtype([('city_start', 'S10'), ('city_end', 'S10'), ('distance', int)])
data_set = np.array(values,dtype=dt)

In [0]:
def all_cities(mdarray):
    """Takes a multi-dimensional array in this format and finds unique cities
    
    array([["A", "A"],
    ["A", "B"]])
    
    """
    cities = {}
    city_set = set(data_set['city_end'])
    for city in city_set:
        cities[city] = ""
    return cities
    
def randomize_city_start(all_cities):
    """Returns a randomized city to start trip"""
    
    return choice(all_cities)

def get_shortest_route(routes):
    """Sort the list by distance and return shortest distance route"""

    route = sorted(routes, key=lambda dist: dist[2]).pop(0)
    return route

  
def greedy_path(starting_city):
    """Select the next path to travel based on the shortest, nearest path"""

    itinerary = []
    cities = all_cities(data_set)
    #starting_city = randomize_city_start(list(cities.keys()))
    #print "starting_city: %s" % starting_city
    cities_visited = {}
    #we want to iterate through all cities once
    count = 1
    while True:
        possible_routes = []
        distance = [] 
       # print( "starting city: %s" % starting_city)
        for path in data_set:
            if starting_city in path['city_start']:
                #we can't go to cities we have already visited
                if path['city_end'] in cities_visited:
                    continue
                else:
                    #print ("path: ", path)
                    possible_routes.append(path)
        
        
        if not possible_routes:
            break
        #append this to itinerary
        #print("possible_route", possible_routes[0:3])
        route = get_shortest_route(possible_routes)
        #print ("Route(%s): %s " % (count, route))
        count += 1
        itinerary.append(route)
        #add this city to the visited city list
        cities_visited[route[0]] = count
        #print ("cities_visited: %s " % cities_visited)
        #reset the starting_city to the next city
        starting_city = route[1]
        #print "itinerary: %s" % itinerary
    for path in data_set:
        if (itinerary[-1]['city_end']==path['city_start'] and itinerary[0]['city_start']==path['city_end']):
           itinerary=itinerary+[path]       
   
    return itinerary

def get_total_distance(complete_itinerary):
    
    distance = sum(z for x,y,z in complete_itinerary)
    return distance

def lowest_simulation(num):
    
    routes = {}
    for i in range(num):
        itinerary = greedy_path()
        distance = get_total_distance(itinerary)
        routes[distance] = itinerary
    shortest_distance = min(routes.keys())
    route = routes[shortest_distance]
    return shortest_distance, route

def main(starting_city):
    """runs everything"""
    dic = {}
    start = []
    final_itinerary=[]
    if len(sys.argv) == 2:
        iterations = int(sys.argv[1])
        print ("Running simulation %s times" % iterations)
        distance, route = lowest_simulation(iterations)
        print ("Shortest Distance: %s" % distance)
        print ("Optimal Route: %s" % route)
    else:
        #print ("All Routes: %s" % data_set)
        itinerary = greedy_path(starting_city)
        #print ("itinerary: %s" % itinerary)
        #print ("Distance: %s" % get_total_distance(itinerary))
    
    total_distance = int(get_total_distance(itinerary))   
    start = itinerary[0][0]
    final_itinerary.append(start)
    for step in itinerary:
      end = step["city_end"]
      
      final_itinerary.append(end)
    #print(final_itinerary)
    
    finalitnerary = ' - '.join( map(str, final_itinerary))
    
    #save starting city and distance in a dictionary
    dic[finalitnerary] = total_distance
    #print(dic)
    return dic

#set times of running
running_times = 500


#generate starting point randomly
def starting():
  cities = all_cities(data_set)
  starting_point = []
  for i in range(running_times):
    start_city = randomize_city_start(list(cities.keys()))
    starting_point.append(start_city)  
  return starting_point

 #parellization
if __name__ == '__main__':
  starting_city = starting() 
  with Pool(running_times) as p:
    dis = p.map(main,starting_city)



In [0]:
#change the width of column to show the whole itinerary
pd.options.display.max_colwidth = 10000

In [0]:
#print the shortest route of different starting city and the distance
cols=["route","distance"]
df = pd.DataFrame(columns=cols)
master_df = pd.DataFrame(columns=cols)
for i in dis:
  df["route"]=i.keys()
  df["distance"]=i.values()
  master_df= master_df.append(df)
  
route = master_df["route"]
city_number = route.nunique()
print(city_number," cities have been selected as starting point.")


50  cities have been selected as starting point.


In [0]:
shortest_route = master_df[master_df["distance"] == min(master_df["distance"])]
shortest_route.drop_duplicates(inplace=True)
route_final = shortest_route["route"]
distance_final = shortest_route["distance"]

print ("Shortest Distance: %s" % distance_final.values)
print ("Optimal Route: %s" % route_final.values)

Shortest Distance: [19617]
Optimal Route: ["b'Salem' - b'Olympia' - b'Boise' - b'Helana' - b'Salt Lake ' - b'Denver' - b'Cheyenne' - b'Pierre' - b'Lincoln' - b'Topeka' - b'Jefferson ' - b'Springfiel' - b'Indianapol' - b'Frankfort' - b'Columbus' - b'Charleston' - b'Richmond' - b'Annapolis' - b'Dover' - b'Trenton' - b'Harrisburg' - b'Albany' - b'Hartford' - b'Providence' - b'Boston' - b'Concord' - b'Montpelier' - b'Augusta' - b'Lansing' - b'Madison' - b'Saint Paul' - b'Des Moines' - b'Oklahoma C' - b'Little Roc' - b'Jackson' - b'Baton Roug' - b'Montgomery' - b'Atlanta' - b'Columbia' - b'Raleigh' - b'Nashville' - b'Tallahasse' - b'Austin' - b'Santa Fe' - b'Phoenix' - b'Carson Cit' - b'Sacramento' - b'Bismarck' - b'Juneau' - b'Honolulu' - b'Salem'"]


In [0]:
#print the itinerary of shortest distance to a file
outF = open("myOutFile.txt", "w")
for ind, line in shortest_route.iterrows():
  # write line to output file
  outF.write(str(line))
  outF.write("\n")
  #print(line)
outF.close()